### 프롬프트 추론 데이터 수집 3차 스크립트(예시)

In [ ]:
import fitz
import re
test_file = r'C:\Users\DANIK\github_dk\etc\data\평가원.2020.152.문제.pdf' 

In [ ]:
doc = fitz.open(test_file)
for page in doc:
    blocks = page.get_text("blocks")
    for block in blocks:
        print(block)

---

0415 테스트

In [1]:
import fitz
import re
from collections import defaultdict
test_file = r'C:\Users\DANIK\github_dk\etc\data\평가원.2020.152.문제.pdf' 
doc = fitz.open(test_file)


In [2]:

num_patten = re.compile("([0-9]+)(\.)")
option_patten = re.compile('(\[)+(\d)+(.\S)+(\d)+(\])')

result = defaultdict()
for page in doc:
    page = page.get_text("blocks")
    # blocks(list) : page
    
    text = ''
    
    for line in page:
        # line
        # (x0, y0, x1, y1, "lines in the block", block_no, block_type)
        # line[-1] == 0:  # Text
        # line[-3] # "lines in the block"
        if line[-2] <=1 :             
            continue

        if line[-1] == 0 :  # Text
            current_line = line[-3]  # "lines in the block"
            current_line = re.sub('(\S)*(\[[0-9]+점])','',current_line)  # \\[3점]\\ 텍스트 제거
            current_line = re.sub('(\*)+(.+)','',current_line)  # \\* morphology: 형태\\ 제거
            current_line = current_line.replace('\xad','-')  # \\* morphology: 형태\\ 제거
            
            is_pass = True if re.search('학년도|\확인 사항|이제 듣기 문제가 끝났습니다.|답을 하시기|답안지의|로그인 필요 없는 학습 자료 무료 제공 사이트!|로그인 필요 없는 학습 자료 무료 제공 사이트!|www.LegendStudy.com',current_line) else False
            
            current_line_is_query_num = True if num_patten.search(current_line[:3]) else False
            current_line_is_query_common = True if option_patten.search(current_line) else False
            
            # 현재 라인에서 숫자 발견 or [] 발견했으면,            
            if current_line_is_query_num or is_pass or current_line_is_query_common:
                if option_patten.search(text) and '다음을 듣고,' not in text :
                    number_s,number_e = re.findall('\d+',text[:8])
                    number_s,number_e = int(number_s),int(number_e)
                                        
                    for key in range(number_s,number_e+1):
                        result[key] = {'query': text}
                    
                    text = ''
                    
                if num_patten.search(text) : ## 여기서 elif 로 수정?
                    s_ind,e_ind = num_patten.search(text).span()
                    key = int(text[s_ind:e_ind].replace('.','').strip())
                    query = text[e_ind:]
                    if key in result.keys() :
                        result[key]['query'] +=query
                    else :
                        result[key]=({'query': query})
                    
                    text =''

                if is_pass :
                    continue
            
            text += current_line.replace('\n','')
            
        else:
            text +="<!image>"
    else :

        if num_patten.search(text) : ## 여기서 elif 로 수정?
            s_ind,e_ind = num_patten.search(text).span()
            key = int(text[s_ind:e_ind].replace('.','').strip())
            query = text[e_ind:]
            if key in result.keys() :
                result[key]['query'] +=query
            else :
                result[key]=({'query': query})
            
            text =''


In [3]:
result

defaultdict(None,
            {1: {'query': ' 대화를 듣고, 여자의 마지막 말에 대한 남자의 응답으로 가장 적절한 것을 고르시오.① I already did, but I couldn’t find it.② Okay, I’ll wait in the living room.③ You need to fix the cellphone.④ I called the Lost and Found.⑤ Wi–Fi is not available here.'},
             2: {'query': ' 대화를 듣고, 남자의 마지막 말에 대한 여자의 응답으로 가장 적절한 것을 고르시오. ① Visiting a sunflower festival would be nice.② You can’t send her a birthday card.③ Your kindness is always repaid.④ I’ll wrap the flowers for free.⑤ White roses are nice at this time of year.'},
             3: {'query': ' 다음을 듣고, 여자가 하는 말의 목적으로 가장 적절한 것을 고르시오.① 축제 관련자 안전 교육 참석을 공지하려고② 정해진 장소에서 활동할 것을 요청하려고③ 다양한 공연을 준비할 것을 독려하려고④ 동아리 담당 교사 변경을 안내하려고 ⑤ 적극적인 동아리 활동을 부탁하려고'},
             4: {'query': ' 대화를 듣고, 남자의 의견으로 가장 적절한 것을 고르시오.① 땀을 많이 흘린 후에는 수분 보충이 중요하다.② 운동 전 음식 섭취가 운동을 위한 힘을 준다.③ 운동은 땀을 흘릴 만큼 충분히 해야 한다.④ 달리기 전 충분한 준비 운동을 해야 한다.⑤ 꾸준한 운동이 건강한 신체를 만든다.'},
             5: {'query': ' 대화를 듣고, 두 사람의 관계를 가장 잘 나타낸 것을 고르시오.① 구직자 - 채용 담당 직원② 소설가 - 영화감독 